### ペア多数決 (pairwise majority rule) による選好プロファイルの集約
- あり得るすべての選択肢のペア $\forall o_1, o_2 \in O$ について，
  各投票者は自身の選好順序において順位が高いほうの選択肢に 1票だけ投票する。
  
- $\#(o_1 \succ o_2)$によって，$o_1 \succ_i o_2$となっている投票者
  の人数を表現するとき，集計結果に応じて，

  $$
  \begin{aligned}
      o_1 \succ_W o_2 \quad \text{if} \quad \#(o_1 \succ o_2) > \#(o_2 \succ o_1)  \\
      o_1 \sim_W  o_2 \quad \text{if} \quad \#(o_1 \succ o_2) = \#(o_2 \succ o_1) \\
      o_1 \prec_W o_2 \quad \text{if} \quad \#(o_1 \succ o_2) < \#(o_2 \succ o_1) 
  \end{aligned}
  $$

  のいずれかの順序を選ぶ。

- 選択肢$o_1, o_2, \ldots o_m$の全体についての一連の順序を決定するには，
  選好の推移性に従う。

- 選好の推移性だけでは，順序が循環的となってしまう選好プロファイルへの対処については
  コンドルセ-ヤング・ルール `ch10_condorcet_young_rule.ipynb` を参照。

In [13]:
import itertools as it
import numpy as np
# from scipy.stats import randint
import scipy.stats
from IPython.display import display_markdown

# alternatives = ('a', 'b', 'c', 'd') 
alternatives = ('a', 'b', 'c') 
prefs = np.array(tuple(it.permutations(alternatives)))

if False:
    num_voters = 100 #5 #6
    profile_idx = scipy.stats.randint.rvs(0, len(prefs), size=num_voters)
    profile = prefs[profile_idx]

elif True:
    # tally = {('a', 'b', 'c'): 30,
    #          ('a', 'c', 'b'): 56,
    #          ('c', 'a', 'b'): 7,
    #          ('c', 'b', 'a'): 28,
    #          ('b', 'c', 'a'): 62,
    #          ('b', 'a', 'c'): 15}

    # tally = {('a', 'b', 'c'): 17,
    #          ('a', 'c', 'b'): 18,
    #          ('c', 'a', 'b'): 8,
    #          ('c', 'b', 'a'): 9,
    #          ('b', 'c', 'a'): 26,
    #          ('b', 'a', 'c'): 0}

    # tally = {('a', 'b', 'c'): 18,
    #          ('a', 'c', 'b'): 19,
    #          ('c', 'a', 'b'): 8,
    #          ('c', 'b', 'a'): 8,
    #          ('b', 'c', 'a'): 25,
    #          ('b', 'a', 'c'): 0}

    tally = {('a', 'b', 'c'): 37,
             ('a', 'c', 'b'): 53,
             ('c', 'a', 'b'): 12,
             ('c', 'b', 'a'): 21,
             ('b', 'c', 'a'): 65,
             ('b', 'a', 'c'): 10}

    profile = np.array(tuple(it.chain.from_iterable(it.repeat(p, v) for p, v in tally.items())))
    num_voters = len(profile) 

else:
    profile = prefs
    num_voters = len(profile) 

pref_str = lambda p: r' $\succ$ '.join(p)
display_markdown('  \n'.join(f'voter {i}: &emsp;' + pref_str(p) for i, p in enumerate(profile) if i < 5)
+ '  \n' + r'$\quad \vdots$', raw=True)


def is_preferred_to(pref, x, y):
    """
    選好順序 pref において x が y よりも上位にあれば True,
    そうでなければ False を返す。
    """
    pos_x = np.flatnonzero(pref == x)[0]
    pos_y = np.flatnonzero(pref == y)[0]
    return pos_x < pos_y

pairwise_tally = {
    cp: np.count_nonzero([is_preferred_to(p, *cp) for p in profile])
        for cp in it.combinations(alternatives, 2)}  

display_markdown('  \n'.join(
    f'$\#$({pref_str(c)}) == {s}, &emsp; ' + 
    f'$\#$({pref_str(c[::-1])}) == {num_voters - s}' 
        for c, s in pairwise_tally.items()), raw=True)

winner = {a: True for a in alternatives}

for a in alternatives:
    for (x, y),s in pairwise_tally.items():
        if ((x == a and s < num_voters - s) or
            (y == a and num_voters - s < s)):
                winner[a] = False

print(f'p.m.r. winner: {winner}') # p.m.r.: pairwise majority rule 

voter 0: &emsp;a $\succ$ b $\succ$ c  
voter 1: &emsp;a $\succ$ b $\succ$ c  
voter 2: &emsp;a $\succ$ b $\succ$ c  
voter 3: &emsp;a $\succ$ b $\succ$ c  
voter 4: &emsp;a $\succ$ b $\succ$ c  
$\quad \vdots$

$\#$(a $\succ$ b) == 102, &emsp; $\#$(b $\succ$ a) == 96  
$\#$(a $\succ$ c) == 100, &emsp; $\#$(c $\succ$ a) == 98  
$\#$(b $\succ$ c) == 112, &emsp; $\#$(c $\succ$ b) == 86

p.m.r. winner: {'a': True, 'b': False, 'c': False}


#### 解説
- 以下の解説のセルを正常に実行するには Notebook の最初から順にセルを実行しておく必要がある。
- 選好プロファイルの生成については，`ch10_plurality_voting.ipynb` を参照。

配列 `x` に対して
 [`numpy.flatnonzero(x)`](https://numpy.org/doc/stable/reference/generated/numpy.flatnonzero.html) 
は値が True となる `x` の要素のインデックスを並べた配列を返す。  

In [14]:
x = np.array([True, False, True])
print('x == ', x)
print('np.flatnonzero(x) ==', np.flatnonzero(x))
pref = np.array(['b', 'c', 'a'])
print("(pref == 'b') ==", pref == 'b')
print("np.flatnonzero(pref == 'b') ==", np.flatnonzero(pref == 'b'))
print("(pref == 'c') ==", pref == 'c')
print("np.flatnonzero(pref == 'c') ==", np.flatnonzero(pref == 'c'))

x ==  [ True False  True]
np.flatnonzero(x) == [0 2]
(pref == 'b') == [ True False False]
np.flatnonzero(pref == 'b') == [0]
(pref == 'c') == [False  True False]
np.flatnonzero(pref == 'c') == [1]


配列 `x` に対して [`np.count_nonzero(x)`](https://numpy.org/doc/stable/reference/generated/numpy.count_nonzero.html) は 評価結果が True となる `x` の要素の個数を返す。

In [15]:
x = np.array([True, True, False, False, False, False])
print('np.count_nonzero(x) ==', np.count_nonzero(x))

np.count_nonzero(x) == 2


`is_preferred_to(p, *cp)` は `is_preferred_to(p, cp[0], cp[1])` と同等な関数呼び出しとなる。

In [16]:
def f(x, y):
    return x + y

t = (1, 2)
print('f(*t) == ', f(*t))

f(*t) ==  3


選択肢の集合 $O$ について， あり得るすべての選択肢のペア $o_1, o_2 \in O$ は
[`it.combinations()`](https://docs.python.org/ja/3/library/itertools.html#itertools.combinations)
で列挙できる。

In [17]:
[cp for cp in it.combinations(alternatives, 2)]

[('a', 'b'), ('a', 'c'), ('b', 'c')]